*Notes:*

-Messages from:  https://t.me/movimientoqanonperu

-Exported the chats using Telegram Lite application

-Scraped the chats on the file "peru.py," and created a csv with all the messages called "mensajes.csv"

-Next Step: Analyzing the messages 

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 300)

/Users/biancapallaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Importing the messages and naming the columns

In [2]:
df = pd.read_csv("mensajes_old.csv", header=None)
df.columns = ["date_time", "fowarded", "user", "message", "in_reply"]
df.tail(10)

,date_time,fowarded,user,message,in_reply
149139,NaN,True,QAnon Map⚡️ 07.05.2021 09:59:53,BQQM 😁🔥🔥🔥🔥🔥🐸@QAnonMap 💀,NaN
149140,NaN,NaN,NaN,Media not included,NaN
149141,07.05.2021 12:59:12,NaN,Raissa Ponte Mejía,"Un “científico” sueco??, plantea alimentarnos con carne humana para solucionar el cambio climático .... 🤮🤮https://www.standard.co.uk/news/world/scientist-suggests-eating-human-meat-to-tackle-cl...",NaN
149142,NaN,True,GOLPE EN RETROCESO con los planes oscuros .... 07.05.2021 12:54:55,"Un “científico” sueco??, plantea alimentarnos con carne humana para solucionar el cambio climático .... 🤮🤮https://www.standard.co.uk/news/world/scientist-suggests-eating-human-meat-to-tackle-cl...",NaN
149143,07.05.2021 13:00:02,NaN,Vilma M,Es demasiado,In reply to this message
149144,07.05.2021 13:00:17,NaN,Christian,"Mujeres progre: ""Los hombres tienen que dejar de vernos como objetos sexuales!!""",NaN
149145,NaN,True,John Acquaviva 07.05.2021 12:50:17,"Mujeres progre: ""Los hombres tienen que dejar de vernos como objetos sexuales!!""",NaN
149146,NaN,NaN,NaN,Media not included,NaN
149147,07.05.2021 13:01:29,NaN,Vilma M,Media not included,In reply to this message
149148,NaN,NaN,NaN,Media not included,NaN


Elminating the "in_reply" column which I won't use for this analysis:

In [3]:
df = df.drop(labels='in_reply', axis=1)
df.head()

,date_time,fowarded,user,message
0,NaN,NaN,Service message,25 April 2020
1,NaN,NaN,Service message,Q AnonPeru 🇵🇪 converted a basic group to this supergroup «QuanonPeru 🇵🇪»
2,NaN,NaN,Service message,3 May 2020
3,NaN,NaN,Service message,Prometeo 🇵🇪 changed group title to «Q AnonPeru 🇵🇪»
4,NaN,NaN,Service message,5 May 2020


Getting rid of "Service message" + "Media not included" + forwareded messages (that I imported twice because you have the user who sent the message to this group chat and the user who wrote the message in the first place and that might not be a member of the groups)

In [4]:
df_no_sm = df[(df.user != 'Service message') & (df.message != 'Media not included') & (df.fowarded != True)]

Looking at the types. I will be changing the date type later on in this notebook.

In [5]:
df_no_sm.dtypes

date_time    object
fowarded     object
user         object
message      object
dtype: object

Looking at the number of columns and rows

In [6]:
df_no_sm.shape

(71192, 4)

Eliminating the nulls with blank spaces because nltk and count vectorizer don't work with null values

In [7]:
df_no_sm.message = df_no_sm.message.fillna('')

/Users/biancapallaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [8]:
df_no_sm.head()

,date_time,fowarded,user,message
5,05.05.2020 22:44:14,NaN,Mauricio Triana Villalón Salseroporexcelencia,Acá estoy Rey
6,05.05.2020 22:45:25,NaN,Prometeo 🇵🇪,👍
7,05.05.2020 22:49:28,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/JuE7V6-2cyw
8,05.05.2020 22:50:24,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/urxNPMIQNAg
9,05.05.2020 22:50:55,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/w-iqZ4Ou6v8


Count vectorizer in Spanish. Found this guide: https://pybonacci.org/2015/11/24/como-hacer-analisis-de-sentimiento-en-espanol-2/

In [9]:
!pip install nltk

You should consider upgrading via the '/Users/biancapallaro/.pyenv/versions/3.8.2/bin/python3.8 -m pip install --upgrade pip' command.


Using the code from the webpage above. 
At first, I tried using the stemmer but the results weren’t accurate. So, I eliminated the stemmer by commenting the code.

In [10]:
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
spanish_stopwords = stopwords.words('spanish')
#stemmer = SnowballStemmer('spanish')
non_words = list(punctuation)
non_words.extend(['¿', '¡'])
non_words.extend(map(str,range(10)))
#stemmer = SnowballStemmer('spanish')
#def stem_tokens(tokens, stemmer):
    #stemmed = []
    #for item in tokens:
        #stemmed.append(stemmer.stem(item))
    #return stemmed
def tokenize(text):
    text = ''.join([c for c in text if c not in non_words])
    tokens =  word_tokenize(text)
    # stem
    #try:
        #stems = stem_tokens(tokens, stemmer)
   # except Exception as e:
        #print(e)
        #print(text)
        #stems = ['']
    #return stems
    return tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/biancapallaro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/biancapallaro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Importing CountVectorizer. For more information look at: https://investigate.ai/text-analysis/counting-words-with-scikit-learns-countvectorizer/

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

# Make a vectorizer
vectorizer = CountVectorizer(
                analyzer = 'word',
                tokenizer = tokenize,
                lowercase = True,
                stop_words = spanish_stopwords,
                min_df=20)

# Learn and count the words in df.content
matrix = vectorizer.fit_transform(df_no_sm.message)

# Convert the matrix of counts to a dataframe
words_df = pd.DataFrame(matrix.toarray(),
                        columns=vectorizer.get_feature_names())

In [12]:
words_df

,abajo,abandona,abandonar,abc,aberrante,abierta,abiertamente,abiertas,abierto,abiertos,abogada,abogado,abogados,abortados,aborto,abortos,about,abraham,abran,abrazo,abrazos,abre,abren,abrieron,abril,abrir,abrió,absoluta,absolutamente,absoluto,absurdas,absurdo,abuelos,abusar,abuso,abusos,ac,aca,acaba,acabado,acaban,acabar,acabará,acabe,acabo,acabó,acaso,acceder,acceso,accidente,...,📲,📹,🔔,🔗,🔥,🔴,🔵,🔻,😁,😂,😂😂,😂😂😂,😂😂😂😂,😃,😅,😉,😊,😎,😑,😒,😔,😞,😡,😡😡😡,😢,😭,😱,😳,🙂,🙄,🙌,🙏,🙏🏻,🙏🏼,🙏🙏,🙏🙏🙏,🚨,🛑,🤔,🤔🤔,🤗,🤣,🤣🤣🤣,🤦‍♂️,🤬,🤮,🤷‍♀,🧐,🧠,🧩
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71187,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71188,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71189,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71190,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Showing the top 11 most common words

In [13]:
words_df.sum().T.sort_values(ascending=False).head(11) 

si        6808
q         6788
trump     4152
the       3929
perú      3459
🇵🇪        3158
ser       3128
covid     3094
vacuna    2911
anon      2760
gente     2674
dtype: int64

Creating a dataframe with the most common words

In [14]:
common_words = words_df.sum().T.sort_values(ascending=False).to_frame(name='count_words')

In [15]:
common_words.head(100)

,count_words
si,6808
q,6788
trump,4152
the,3929
perú,3459
🇵🇪,3158
ser,3128
covid,3094
vacuna,2911
anon,2760


In [16]:
common_words = common_words.reset_index()

In [17]:
common_words = common_words.rename(columns={'index': 'common_word', 'count_words': 'count'})
common_words.head()

,common_word,count
0,si,6808
1,q,6788
2,trump,4152
3,the,3929
4,perú,3459


How many times and when the following words appear? the president's name / Trump / Facebook / 

In [18]:
common_words[common_words['common_word'].str.contains("vizcarra")]

,common_word,count
690,vizcarra,219
4840,vizcarrata,31


In [19]:
df_no_sm[df_no_sm["message"].str.contains("Vizcarra")]

,date_time,fowarded,user,message
940,16.07.2020 17:38:46,NaN,Deleted Account,Ahora que se va Vizcarra que cosas no firmara perjudicandonos ....
1579,28.07.2020 14:00:29,NaN,cindy fernandez,"Y terminó de hablar Vizcarra , pero aún no sale nada de la marcha de apoyo al cds"
2603,04.08.2020 14:29:26,NaN,Jorge Montalvo,"Vizcarra esta hace tiempo bajo las ordenes del Cabal, la quiebra de la economia, el ingreso de la pandemia. la entrega de fondos a inglaterra por 30 mil millones via contrato estado - estado dine..."
2719,05.08.2020 10:54:41,NaN,Julie Pinzas De La Mata,Y ese dinero se lo repartirá Vizcarrata y secuaces... el pueblo? Ni hablar
2812,06.08.2020 12:34:36,NaN,Maricarmen,Por el préstamo de millones que ha recibido d FMI muy clarito el accionar de Vizcarra
3171,09.08.2020 15:02:10,NaN,Bell,DenunciaAVizcarrahttps://www.facebook.com/162529584403844/posts/638895196767278/?sfnsn=mo&d=n&vh=e
3469,12.08.2020 22:08:14,NaN,Alirio Valero,También pienso lo mismo.Esta semana sera el domingo la otra semana saldrá diciendo Vizcarra que no es efectivo solo el domingo que tendrá que ser total la 40tna. Otra vez presos
3973,15.08.2020 12:01:59,NaN,Maricarmen,"Me da que pensar los casos de violencia que hubo hace poco y por eso Vizcarra tomó las últimas medidas drásticas, no será que tenemos pequeños antifas pagados?"
4623,17.08.2020 18:49:10,NaN,Nena Delboy,sistema del cabal seguro 👆🏻Vizcarra alienta a invertir en bitcoins
5285,19.08.2020 02:03:30,NaN,Maricarmen,100 millones de soles a las Farmacéuticas regala Vizcarra


In [20]:
common_words[common_words['common_word'].str.contains("sagasti")]

,common_word,count
957,sagasti,168


In [21]:
df_no_sm[df_no_sm["message"].str.contains("Sagasti")]

,date_time,fowarded,user,message
29261,16.11.2020 15:00:56,NaN,Chano Gálvez,Francisco Sagasti Sendero luminoso nuevo presidente es cierto????????
29305,16.11.2020 15:20:45,NaN,👽,Que opinan de Francisco Sagasti creen que seguirá la agenda de Viscarra?
29392,16.11.2020 16:12:04,NaN,Christian,A trollear el en vivo a favor de Sagasti de El Comercio
29397,16.11.2020 16:22:58,NaN,Christian,Si están oyendo a Sagasti empieza su discurso habla de manifestación pacífica. Mintiendo desde el saqué
29420,16.11.2020 17:23:11,NaN,Christian 16.11.2020 17:22:15,Le tiran botellazos a Sagasti
29528,16.11.2020 20:35:26,NaN,Joyce Gonzalez,"ADMIRADOR DEL MRTA VA CON EL NÚMERO 1 CON EL PARTIDO MORADOPARTIDO MORADO LLEVA COMO NÚMERO 1 A FRANCISCO SAGASTI, ADMIRADOR DE TERRORISTA NESTOR CERPA CARTOLINI.Sagasti fue uno de los rehenes de ..."
29539,16.11.2020 20:48:09,NaN,Emil Sifuentes,Este ladrón y genocida. Dice primo la responsabilidad???? O sea con Sagasti. Será responsable y con Merino???? Ahora vemos quienes.estan detrás de todo. El DEEP STATE peruano. La cabal de Soros en...
29603,16.11.2020 22:14:44,NaN,Jacqueline V,La mano derecha de Sagasti es judío 😖
29615,16.11.2020 22:39:24,NaN,Chano Gálvez,"No compares la financiacion De open society de las campañas de 1° Fujimori, luego Humala Toledo, ppk y la puesta de Vizcarra por el globalismo y ahora Soros pone a Sagasti Títere de Soros y ust..."
29629,16.11.2020 22:49:23,NaN,Chano Gálvez,Soy Peruano radicando en Europa hace 27 años y pertenezco a grupos Q anon europa y se habla de todo pero lkego a Qanon perú y me dicen que no se puede despertar a la gente? La gente tiene que sabe...


In [22]:
common_words[common_words['common_word'].str.contains("trump")]

,common_word,count
2,trump,4152
1309,httpstmetrumpintel,126
6042,realdonaldtrump,24


In [23]:
df_no_sm[df_no_sm["message"].str.contains("trump")]

,date_time,fowarded,user,message
420,22.06.2020 20:30:57,NaN,Alvaro Ernesto,Solamente he visto que Q orienta a la gente a creer en trump y en el cabal
915,16.07.2020 10:45:19,NaN,Edu Eyland +,"Trump da órdenes a los hospitales de ""saltarse a los CDC"" y enviar los datos acerca del Covid directamente al Instituto de Salud. En otras palabras: los propagadores del miedo han quedado desartic..."
1798,31.07.2020 04:06:55,NaN,Franco Alfaro,"""Las elecciones del 2020 estarán totalmente amañadas si se permite el voto por correo"": Donald Trump. El presidente Trump ha compartido una investigación de un medio local que probaba la facilidad..."
2079,01.08.2020 21:33:47,NaN,Joe Parker🎩🎭,Y el hermano de trump tambien ha tuiteado cosas sobre kenedy
2179,02.08.2020 09:26:33,NaN,Gisselle,Hola hermosa familia hace ya algun tiempo os decia que pronto os traeria informacion de mi conexion con mi informanteMuy muy cercano a todo esto por respecto y seguridad no puedo desvelarlesEspero...
...,...,...,...,...
147774,05.05.2021 15:15:05,NaN,Glenda Pérez,La Junta de Supervisión de Facebook confirma la suspensión de Donald Trump de la plataformahttps://cnnespanol.cnn.com/2021/05/05/facebook-suspension-donald-trump-de-la-plataforma-trax/
148028,05.05.2021 19:50:37,NaN,javier,Private Miami school strongly advices their teachers NOT to take any genoccine-vaccin and states: “We will no longer employ anyone who got any Covid-l9 vaccine.’’ ...
148736,06.05.2021 19:05:27,NaN,Alejandro Garcia,"🚨🚨EXCLUSIVO: El presidente Trump lanzó el martes una plataforma de comunicaciones, que eventualmente le dará la capacidad de comunicarse directamente con sus seguidores, después de meses de ser pr..."
148928,07.05.2021 07:26:57,NaN,javier,👀👀👀👀 ‘’They had no control over the machines’’. ====================🍿 https://t.me/trumpintel


In [24]:
common_words[common_words['common_word'].str.contains("facebook")]

,common_word,count
178,facebook,595
791,httpsbitlyqduxqwfacebook,199
1078,httpswwwfacebookcomposts,148
1101,httpsmfacebookcomstoryphpstoryfbidid,146
1253,httpswwwfacebookcompostssfnsnmo,131
1715,httpswwwfacebookcomeuskalnewscom🕊️twitter,97
1726,httpstmeeuskalnews📘facebook,97
1917,httpsmfacebookcomstoryphpstoryfbididsfnsnmo,88
3138,httpswwwfacebookcomwatchv,52
3628,httpswwwfacebookcomvideos,44


In [25]:
df_no_sm[df_no_sm["message"].str.contains("Facebook")].head(50)

,date_time,fowarded,user,message
407,21.06.2020 21:26:47,NaN,Wilson C Gómez,"En Facebook y YouTube estánbloqueando cuentas que no se alinean con este sistema corrupto, cuando habla de verdades o publicas verdadeste bloquean,hoy a las 8 y 17 dela noche, e sido bloqueado por..."
1098,23.07.2020 00:09:31,NaN,Nena Delboy,"Munición para las MEMES Wars.Listo para ser usado en Twitter, Facebook o donde gustéis. 😆MEME AMMUNITION:* Big Pharma — https://mega.nz/folder/9MplDKrS#2ErTJWcfFQA3scFeGu02hA* Mueller — https://me..."
1277,26.07.2020 01:21:35,NaN,Deleted Account,"PARQUE INFANTIL SATÁNICO PARA NIÑOSCinecittà World: el parque infantil con entrada en forma de Moloch. Tormenta en la web: ""Satanistas"" En tiempos como estos, nada sale bien si hablamos de ""satani..."
1479,27.07.2020 16:23:38,NaN,Rainbow,En nuestras historias de Facebook durará 24 horas
1528,28.07.2020 09:21:50,NaN,Edwin Alejandro Flores Alava,"Buenos día con todos, me es imposible asistir hoy amigos, pero me gustaría compartir transmisión en vivo en mi Facebook espero transmitan en la página de QAnon y poder apoyar de esta manera a que ..."
1912,31.07.2020 20:21:33,NaN,Rainbow,"Gracias Dulce María es un sentir inexplicable entre paz y nostalgia, hace años vengo compartiendo temas en una comunidad que formé en Facebook Mujer Plena Mente millonaria , espero me visiten , co..."
2541,04.08.2020 10:13:43,NaN,S.C.A,Por favor lean hacia atrás diez o doce publicaciones y esta la misma Esto ya parece Facebook 😡😡😡 se supone que los que enyrsmos en este grupo es para recibir información contrastada al respecto de...
3034,08.08.2020 10:55:29,NaN,joseph Q,"Facebook elimina el grupo de conspiración QAnon con 200,000 miembros - BBC Newshttps://www.bbc.com/news/technology-53692545"
3288,10.08.2020 23:56:04,NaN,SHIRLEY,Agentes Flas - NO EXISTE PROEBAS RAPIDAS DE CVD 19 ES UN ENGAÑO | Facebookhttps://m.facebook.com/story.php?story_fbid=296502721785608&id=100042776475404
3373,12.08.2020 11:07:40,NaN,SHIRLEY,🚩La opinión de GS en la administración Obama_... - Aepoket Hernández | Facebookhttps://m.facebook.com/story.php?story_fbid=1546967192139773&id=100004795205538


Looking at who is talking the most

In [26]:
df_no_sm.user.value_counts().head(10)

Patriota 🇵🇪         6384
Vanesa Ch L         2982
Seba Horton         1149
Tintin               955
Deleted Account      834
Juan C               822
Alii                 802
Maricarmen           792
Rafael◉              717
Libre o muerto!!     707
Name: user, dtype: int64

Converting the date to date format

In [27]:
df_no_sm['date_column'] = pd.to_datetime(df_no_sm['date_time'], format='%d.%m.%Y %H:%M:%S', errors='coerce')
df_no_sm.head()

<ipython-input-27-3b3eb82fdf5c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_sm['date_column'] = pd.to_datetime(df_no_sm['date_time'], format='%d.%m.%Y %H:%M:%S', errors='coerce')


,date_time,fowarded,user,message,date_column
5,05.05.2020 22:44:14,NaN,Mauricio Triana Villalón Salseroporexcelencia,Acá estoy Rey,2020-05-05 22:44:14
6,05.05.2020 22:45:25,NaN,Prometeo 🇵🇪,👍,2020-05-05 22:45:25
7,05.05.2020 22:49:28,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/JuE7V6-2cyw,2020-05-05 22:49:28
8,05.05.2020 22:50:24,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/urxNPMIQNAg,2020-05-05 22:50:24
9,05.05.2020 22:50:55,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/w-iqZ4Ou6v8,2020-05-05 22:50:55


Creating a column just for the day (without time)

In [28]:
df_no_sm['day_date'] = df_no_sm.date_column.dt.strftime('%Y-%m-%d')
df_no_sm.head()

<ipython-input-28-42e2242f5a95>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_sm['day_date'] = df_no_sm.date_column.dt.strftime('%Y-%m-%d')


,date_time,fowarded,user,message,date_column,day_date
5,05.05.2020 22:44:14,NaN,Mauricio Triana Villalón Salseroporexcelencia,Acá estoy Rey,2020-05-05 22:44:14,2020-05-05
6,05.05.2020 22:45:25,NaN,Prometeo 🇵🇪,👍,2020-05-05 22:45:25,2020-05-05
7,05.05.2020 22:49:28,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/JuE7V6-2cyw,2020-05-05 22:49:28,2020-05-05
8,05.05.2020 22:50:24,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/urxNPMIQNAg,2020-05-05 22:50:24,2020-05-05
9,05.05.2020 22:50:55,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/w-iqZ4Ou6v8,2020-05-05 22:50:55,2020-05-05


Creating a column just for the month

In [29]:
df_no_sm['month_date'] = df_no_sm.date_column.dt.strftime('%m-%Y')
df.head()

<ipython-input-29-253188885515>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_sm['month_date'] = df_no_sm.date_column.dt.strftime('%m-%Y')


,date_time,fowarded,user,message
0,NaN,NaN,Service message,25 April 2020
1,NaN,NaN,Service message,Q AnonPeru 🇵🇪 converted a basic group to this supergroup «QuanonPeru 🇵🇪»
2,NaN,NaN,Service message,3 May 2020
3,NaN,NaN,Service message,Prometeo 🇵🇪 changed group title to «Q AnonPeru 🇵🇪»
4,NaN,NaN,Service message,5 May 2020


Before moving forward, I want to know when did most users join the chat. So, I am going back to the original df that includes the "Service messages."

In [30]:
df['date_column'] = pd.to_datetime(df['date_time'], format='%d.%m.%Y %H:%M:%S', errors='coerce')
df.head()

,date_time,fowarded,user,message,date_column
0,NaN,NaN,Service message,25 April 2020,NaT
1,NaN,NaN,Service message,Q AnonPeru 🇵🇪 converted a basic group to this supergroup «QuanonPeru 🇵🇪»,NaT
2,NaN,NaN,Service message,3 May 2020,NaT
3,NaN,NaN,Service message,Prometeo 🇵🇪 changed group title to «Q AnonPeru 🇵🇪»,NaT
4,NaN,NaN,Service message,5 May 2020,NaT


Creating a day column in the original df

In [31]:
df['day_date'] = df.date_column.dt.strftime('%Y-%m-%d')
df.head()

,date_time,fowarded,user,message,date_column,day_date
0,NaN,NaN,Service message,25 April 2020,NaT,NaN
1,NaN,NaN,Service message,Q AnonPeru 🇵🇪 converted a basic group to this supergroup «QuanonPeru 🇵🇪»,NaT,NaN
2,NaN,NaN,Service message,3 May 2020,NaT,NaN
3,NaN,NaN,Service message,Prometeo 🇵🇪 changed group title to «Q AnonPeru 🇵🇪»,NaT,NaN
4,NaN,NaN,Service message,5 May 2020,NaT,NaN


Creating a month column in the original df

In [32]:
df['month_date'] = df.date_column.dt.strftime('%m-%Y')
df.head()

,date_time,fowarded,user,message,date_column,day_date,month_date
0,NaN,NaN,Service message,25 April 2020,NaT,NaN,NaN
1,NaN,NaN,Service message,Q AnonPeru 🇵🇪 converted a basic group to this supergroup «QuanonPeru 🇵🇪»,NaT,NaN,NaN
2,NaN,NaN,Service message,3 May 2020,NaT,NaN,NaN
3,NaN,NaN,Service message,Prometeo 🇵🇪 changed group title to «Q AnonPeru 🇵🇪»,NaT,NaN,NaN
4,NaN,NaN,Service message,5 May 2020,NaT,NaN,NaN


In [33]:
df.message = df.message.fillna('')

Note: "Invited" means "joined the group"

In [34]:
df_invited = df[df["message"].str.contains("invited")]
df_invited.head(5)

,date_time,fowarded,user,message,date_column,day_date,month_date
98,26.05.2020 19:49:54,NaN,Service message,Castañeda invited Caren Castañeda,2020-05-26 19:49:54,2020-05-26,05-2020
351,18.06.2020 17:46:15,NaN,Service message,Deleted invited Deleted Account,2020-06-18 17:46:15,2020-06-18,06-2020
379,19.06.2020 12:15:32,NaN,Service message,Deleted invited Deleted Account,2020-06-19 12:15:32,2020-06-19,06-2020
380,19.06.2020 12:15:32,NaN,Service message,Evelyn invited Evelyn,2020-06-19 12:15:32,2020-06-19,06-2020
391,20.06.2020 15:02:30,NaN,Service message,Verónica Isabel Perú invited Verónica Isabel Perú,2020-06-20 15:02:30,2020-06-20,06-2020


The day most people joined the group

In [35]:
df_invited.day_date.value_counts()

2021-02-24    287
2021-01-08    124
2021-02-28     91
2021-03-17     83
2021-03-07     82
2021-03-05     81
2021-03-11     76
2021-04-11     73
2021-03-06     73
2020-12-07     69
2021-04-28     68
2021-03-24     65
2021-04-06     64
2021-03-25     64
2021-05-03     63
2021-04-10     63
2021-01-10     61
2021-04-04     60
2021-02-16     58
2021-03-02     57
2021-04-16     55
2021-03-16     54
2021-04-03     54
2021-04-07     51
2020-11-17     50
2021-02-17     47
2021-04-12     46
2021-01-09     45
2021-04-24     43
2021-02-14     42
2021-04-19     41
2021-04-20     40
2021-04-15     40
2021-03-31     37
2020-11-13     37
2021-05-02     37
2021-02-15     36
2020-11-14     35
2021-03-26     35
2021-04-25     35
2020-08-18     35
2021-03-30     33
2021-01-06     33
2021-02-22     33
2021-03-10     32
2021-03-18     30
2020-12-06     30
2021-05-01     28
2020-09-07     27
2021-03-01     27
2021-01-07     27
2021-04-18     26
2020-10-11     26
2021-03-15     26
2020-11-12     26
2020-10-06

In [36]:
joined_day = df_invited.day_date.value_counts()
joined_day = joined_day.to_frame()
joined_day = joined_day.reset_index()
joined_day

,index,day_date
0,2021-02-24,287
1,2021-01-08,124
2,2021-02-28,91
3,2021-03-17,83
4,2021-03-07,82
5,2021-03-05,81
6,2021-03-11,76
7,2021-04-11,73
8,2021-03-06,73
9,2020-12-07,69


In [37]:
import altair as alt
alt.Chart(joined_day).mark_line().encode(
    x='index',
    y='day_date'
)

alt.Chart(...)

The month when most of the people joined the group 

In [38]:
joined = df_invited.month_date.value_counts()
joined = joined.to_frame()
joined = joined.reset_index()
joined

,index,month_date
0,03-2021,1132
1,04-2021,1025
2,02-2021,790
3,01-2021,361
4,12-2020,324
5,11-2020,317
6,09-2020,316
7,10-2020,274
8,08-2020,242
9,05-2021,192


In [39]:
joined_day.to_csv(r'/Users/biancapallaro/Desktop/peru_joined.csv', index = False, header=True)

In [40]:
import altair as alt
alt.Chart(joined).mark_line().encode(
    x='index',
    y='month_date'
)

alt.Chart(...)

The older message I have access to, when the basic group turned to a supergroup called «QuanonPeru 🇵🇪»

In [40]:
df.message.iloc[0]

'25 April 2020'

Group activity: the day most of the messages were sent. For this, we will use the df_no_sm dataframe that has no service messages.

In [41]:
activity = df_no_sm.day_date.value_counts()
activity = activity.to_frame()
activity= activity.reset_index()
activity

,index,day_date
0,2020-11-15,1349
1,2021-01-26,1330
2,2021-01-20,1305
3,2021-02-02,950
4,2021-01-27,837
...,...,...
344,2020-06-02,1
345,2020-06-07,1
346,2020-06-28,1
347,2020-06-27,1


In [42]:
activity.to_csv(r'/Users/biancapallaro/Desktop/peru_activity.csv', index = False, header=True)

In [43]:
import altair as alt
alt.Chart(activity).mark_line().encode(
    x='index',
    y='day_date'
)

alt.Chart(...)

Group activity: the month when most of the messages were sent. 

In [44]:
activity_month = df_no_sm.month_date.value_counts()
activity_month = activity_month.to_frame()
activity_month= activity_month.reset_index()
activity_month

,index,month_date
0,01-2021,16390
1,02-2021,14249
2,11-2020,8522
3,03-2021,7541
4,04-2021,7465
5,08-2020,4009
6,12-2020,3787
7,09-2020,3295
8,10-2020,2657
9,05-2021,2009


In [45]:
import altair as alt
alt.Chart(activity_month).mark_line().encode(
    x='index',
    y='month_date'
)

alt.Chart(...)

Looking at the number of total messages:

In [46]:
df_no_sm.shape

(71192, 7)

Where most fowarded messages come from?
The problem is that the user name from the fowarded messages also contains the date and time of the message. 
To easily solve the issue, I just used regex to take of the numbers and easily make unique users. 

In [47]:
df_fowarded = df[df.fowarded == True]
df_fowarded['user'] = df_fowarded['user'].str.replace('\d+', '')
df_fowarded.user.value_counts().head(10)

<ipython-input-47-1da37d95ad95>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fowarded['user'] = df_fowarded['user'].str.replace('\d+', '')


Maricarmen  .. ::                                                             2467
Noticias Rafapal  .. ::                                                       1383
We The People ⭐️⭐️⭐️🌩🌪🍷🐸- Elite Falling / Elite en Caida / Noticias  .. ::     441
Despertador De La Matrix  .. ::                                                437
Verdades Ofenden @verdadesofenden  .. ::                                       367
Bellisssimaa  .. ::                                                            340
Médicos por la Verdad Argentina 🇦🇷  .. ::                                      317
TierraPura.org | Información sin censura  .. ::                                280
GhostEzra  .. ::                                                               257
Tony Venet  .. ::                                                              248
Name: user, dtype: int64

Searching for the most fowarded messages

In [48]:
df_fowarded = df[(df.fowarded == True) & (df.message != 'Media not included')]
df_fowarded.head(50)

,date_time,fowarded,user,message,date_column,day_date,month_date
140,NaN,True,Qlobal-Change Brasil 🇧🇷 29.05.2020 18:45:30,"Mayo 29, 2020... Aparece nuevo Crop Circle en Wilthshire, Inglaterra.Los detalles, además del simbolismo, son impresionantes.#MaussanTVhttps://www.facebook.com/groups/agrupacionantialienigena/perm...",NaT,NaN,NaN
149,NaN,True,Noticias Rafapal 30.05.2020 02:56:07,"Aquí, el vídeo de la OMS desaprobando el uso de la mascarilla si no se está enfermo. https://m.youtube.com/watch?v=M4olt47pr_o&feature=emb_title",NaT,NaN,NaN
152,NaN,True,INTO THE LIGHT 💫 NETWORK 🌟 30.05.2020 03:26:08,Schicke Maske von Hillary 😂@INTO_THE_LIGHT_NEWS,NaT,NaN,NaN
169,NaN,True,K M 01.06.2020 01:50:17,https://twitter.com/Cesarrios26/status/1267322726437588993?s=09,NaT,NaN,NaN
224,NaN,True,Deleted Account 05.06.2020 21:41:57,https://www.buzzsprout.com/332756/4060805-podcast-daniel-estulin-propaganda-y-el-estado-fascista?client_source=twitter_card&player_type=full_screenhttps://www.buzzsprout.com/332756/4060805-podcast...,NaT,NaN,NaN
304,NaN,True,Mr. Santos News 12.06.2020 11:00:33,NUEVO VIDEO IMPORTANTE!!: https://www.youtube.com/watch?v=1Vnk97FiJR4,NaT,NaN,NaN
339,NaN,True,Deleted Account 17.06.2020 11:25:59,SE BUSCAN MIEMBROS ACTIVOS PARA LOS GRUPOS DE NUEVA HUMANIDAD.Guia uriel luishttps://t.me/joinchat/SkrO2hcekYMkwvogN0P81gNueva humanidad Sevillahttps://t.me/joinchat/HUPjwhzmXmcyWr8gmPPj_ANueva h...,NaT,NaN,NaN
358,NaN,True,ReVelión en la granja comunidad 18.06.2020 11:41:15,"HISTÓRICO!!!!! Finalmente la CONCENTRACIÓN en VALENCIA es LEGALIZADA por el gobierno. QUE SE ENTEREN EN TODA ESPAÑA!!!!!! POR FAVOR, COMPARTAN... Y ACUDAN!!!!!!!!!!!!!!!! DESDE QUE ReVelión en la ...",NaT,NaN,NaN
491,NaN,True,Roliá Athor 01.07.2020 21:46:58,https://youtu.be/BD5uBfuykRg,NaT,NaN,NaN
497,NaN,True,Deleted Account 01.07.2020 23:05:53,No sé si esto sirve de algo pero aquí mi aporte para que lo analicen,NaT,NaN,NaN


In [49]:
df_fowarded.message.value_counts().head(10)

Únete a Q Anon Perú 🇵🇪                                                                                                                                                                                                https://t.me/movimientoqanonperu                                                                                                                                                                                                                                                                        372
Únete a Q Anon Perú 🇵🇪                                                                                                                                                                                                      La Luz ha ganado                                                                                                                                                                                                                                              https://t.me

Creating a dataframe with the urls shared on the group

In [50]:
df_no_sm['url'] = df_no_sm["message"].str.extract(r'(?P<url>https?://[^\s]+)')
df_no_sm.head(25)

<ipython-input-50-dedcd6705858>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_sm['url'] = df_no_sm["message"].str.extract(r'(?P<url>https?://[^\s]+)')


,date_time,fowarded,user,message,date_column,day_date,month_date,url
5,05.05.2020 22:44:14,NaN,Mauricio Triana Villalón Salseroporexcelencia,Acá estoy Rey,2020-05-05 22:44:14,2020-05-05,05-2020,NaN
6,05.05.2020 22:45:25,NaN,Prometeo 🇵🇪,👍,2020-05-05 22:45:25,2020-05-05,05-2020,NaN
7,05.05.2020 22:49:28,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/JuE7V6-2cyw,2020-05-05 22:49:28,2020-05-05,05-2020,https://youtu.be/JuE7V6-2cyw
8,05.05.2020 22:50:24,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/urxNPMIQNAg,2020-05-05 22:50:24,2020-05-05,05-2020,https://youtu.be/urxNPMIQNAg
9,05.05.2020 22:50:55,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/w-iqZ4Ou6v8,2020-05-05 22:50:55,2020-05-05,05-2020,https://youtu.be/w-iqZ4Ou6v8
10,05.05.2020 22:52:21,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/uESfgKcu-gk,2020-05-05 22:52:21,2020-05-05,05-2020,https://youtu.be/uESfgKcu-gk
12,08.05.2020 00:23:42,NaN,Prometeo 🇵🇪,https://www.youtube.com/watch?v=KDdIEKy_xGk&feature=share,2020-05-08 00:23:42,2020-05-08,05-2020,https://www.youtube.com/watch?v=KDdIEKy_xGk&feature=share
13,08.05.2020 00:24:39,NaN,Prometeo 🇵🇪,https://www.youtube.com/watch?v=i8Q-_i5ZZEM&feature=share,2020-05-08 00:24:39,2020-05-08,05-2020,https://www.youtube.com/watch?v=i8Q-_i5ZZEM&feature=share
14,08.05.2020 17:23:20,NaN,Mauricio Triana Villalón Salseroporexcelencia,Discurso de Donald Trump contra el Nuevo Orden mundial,2020-05-08 17:23:20,2020-05-08,05-2020,NaN
17,08.05.2020 17:27:55,NaN,Mauricio Triana Villalón Salseroporexcelencia,Cómo se financia la OMS 👉ONU⚠️🌍 y qué intereses la mueven.,2020-05-08 17:27:55,2020-05-08,05-2020,NaN


In [51]:
urls = df_no_sm[df_no_sm['url'].notnull()]
urls.head()

,date_time,fowarded,user,message,date_column,day_date,month_date,url
7,05.05.2020 22:49:28,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/JuE7V6-2cyw,2020-05-05 22:49:28,2020-05-05,05-2020,https://youtu.be/JuE7V6-2cyw
8,05.05.2020 22:50:24,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/urxNPMIQNAg,2020-05-05 22:50:24,2020-05-05,05-2020,https://youtu.be/urxNPMIQNAg
9,05.05.2020 22:50:55,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/w-iqZ4Ou6v8,2020-05-05 22:50:55,2020-05-05,05-2020,https://youtu.be/w-iqZ4Ou6v8
10,05.05.2020 22:52:21,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/uESfgKcu-gk,2020-05-05 22:52:21,2020-05-05,05-2020,https://youtu.be/uESfgKcu-gk
12,08.05.2020 00:23:42,NaN,Prometeo 🇵🇪,https://www.youtube.com/watch?v=KDdIEKy_xGk&feature=share,2020-05-08 00:23:42,2020-05-08,05-2020,https://www.youtube.com/watch?v=KDdIEKy_xGk&feature=share


Number of urls shared on the chat:

In [52]:
urls.shape

(19606, 8)

Create a new columns with the domains

In [53]:
urls['domain'] = urls['url'].str.extract(r'^(?:.*://)?(?:www\.)?([^:/]*).*$')
urls.head(200)

<ipython-input-53-25bea4726bf4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  urls['domain'] = urls['url'].str.extract(r'^(?:.*://)?(?:www\.)?([^:/]*).*$')


,date_time,fowarded,user,message,date_column,day_date,month_date,url,domain
7,05.05.2020 22:49:28,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/JuE7V6-2cyw,2020-05-05 22:49:28,2020-05-05,05-2020,https://youtu.be/JuE7V6-2cyw,youtu.be
8,05.05.2020 22:50:24,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/urxNPMIQNAg,2020-05-05 22:50:24,2020-05-05,05-2020,https://youtu.be/urxNPMIQNAg,youtu.be
9,05.05.2020 22:50:55,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/w-iqZ4Ou6v8,2020-05-05 22:50:55,2020-05-05,05-2020,https://youtu.be/w-iqZ4Ou6v8,youtu.be
10,05.05.2020 22:52:21,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/uESfgKcu-gk,2020-05-05 22:52:21,2020-05-05,05-2020,https://youtu.be/uESfgKcu-gk,youtu.be
12,08.05.2020 00:23:42,NaN,Prometeo 🇵🇪,https://www.youtube.com/watch?v=KDdIEKy_xGk&feature=share,2020-05-08 00:23:42,2020-05-08,05-2020,https://www.youtube.com/watch?v=KDdIEKy_xGk&feature=share,youtube.com
13,08.05.2020 00:24:39,NaN,Prometeo 🇵🇪,https://www.youtube.com/watch?v=i8Q-_i5ZZEM&feature=share,2020-05-08 00:24:39,2020-05-08,05-2020,https://www.youtube.com/watch?v=i8Q-_i5ZZEM&feature=share,youtube.com
26,09.05.2020 01:05:14,NaN,Mauricio Triana Villalón Salseroporexcelencia,https://youtu.be/zASMqvubHC0,2020-05-09 01:05:14,2020-05-09,05-2020,https://youtu.be/zASMqvubHC0,youtu.be
101,26.05.2020 23:00:08,NaN,Dinelsy,https://youtu.be/1bp2FiFQjcw. Buenas noches tiene que ver este vídeo es un documental echo por la periodista Janet ossenbard es imprescindible que lo VEAN !!!,2020-05-26 23:00:08,2020-05-26,05-2020,https://youtu.be/1bp2FiFQjcw.,youtu.be
118,27.05.2020 18:50:40,NaN,Andres,https://youtu.be/NpWznuEwqcI,2020-05-27 18:50:40,2020-05-27,05-2020,https://youtu.be/NpWznuEwqcI,youtu.be
119,27.05.2020 22:09:21,NaN,Andres,https://m.facebook.com/story.php?story_fbid=231192788328278&id=112477496866475&ref=bookmarks,2020-05-27 22:09:21,2020-05-27,05-2020,https://m.facebook.com/story.php?story_fbid=231192788328278&id=112477496866475&ref=bookmarks,m.facebook.com


Saving the urls:

In [54]:
urls.to_csv(r'/Users/biancapallaro/Documents/Data_Studio/uruguay.csv', index = False, header=True)

The most common domain:

In [55]:
urls.domain.value_counts().head(50)

t.me                        4089
youtu.be                    4055
twitter.com                 1360
facebook.com                1254
youtube.com                 1172
m.facebook.com               463
tierrapura.org               309
loveotv.com                  306
instagram.com                203
bles.com                     175
euskalnews.com               174
eldiestro.es                 156
t.co                         125
rumble.com                   118
lbry.tv                      104
thegatewaypundit.com         100
m.youtube.com                 98
api.parler.com                88
trikooba.com                  87
ejercitoremanente.com         85
divulgaciontotal.com          80
mobile.twitter.com            77
actualidad.rt.com             75
fb.watch                      75
                              66
bitchute.com                  65
bit.ly                        64
telegra.ph                    52
nosmintieron.tv               51
odysee.com                    51
cutt.ly   

The most shared url:

In [56]:
urls.url.value_counts().head(50)

https://t.me/movimientoqanonperu                                                                              1690
https://t.me/despertadordelamatrix                                                                             220
https://t.me/trumpintel                                                                                        122
https://t.me/altavozlibre                                                                                       99
https://t.me/movimientoqanonperu👉🏻                                                                              76
http://t.me/movimientoqanonperu                                                                                 69
https://t.me/movimientoqanonpe                                                                                  52
https://t.me/exponiendolaverdadoculta                                                                           50
https://t.me/PLANDEMIA_MUNDIAL_COVID                                            

Looking at all the telegram links

In [57]:
urls[urls.domain == 't.me'].url.value_counts().head(10)

https://t.me/movimientoqanonperu         1690
https://t.me/despertadordelamatrix        220
https://t.me/trumpintel                   122
https://t.me/altavozlibre                  99
https://t.me/movimientoqanonperu👉🏻         76
http://t.me/movimientoqanonperu            69
https://t.me/movimientoqanonpe             52
https://t.me/exponiendolaverdadoculta      50
https://t.me/PLANDEMIA_MUNDIAL_COVID       47
https://t.me/c/1454197358/169)             42
Name: url, dtype: int64

Clurstering the messages 

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Make a vectorizer
vectorizer = TfidfVectorizer(
                analyzer = 'word',
                tokenizer = tokenize,
                lowercase = True,
                stop_words = spanish_stopwords,
                min_df=50,
                max_df=0.15)

# Learn and count the words in df.content
matrix = vectorizer.fit_transform(df_no_sm.message)

# Convert the matrix of counts to a dataframe
words_df = pd.DataFrame(matrix.toarray(),
                        columns=vectorizer.get_feature_names())

In [59]:
words_df

,abajo,abierta,abiertamente,abierto,abogada,abogado,abogados,aborto,abortos,about,abrazo,abril,abrir,absoluta,absolutamente,absoluto,abuso,abusos,aca,acaba,acaban,acabar,acabo,acabó,acaso,acceso,acciones,acción,aceptar,aceptarme,acerca,acontecimientos,activa,actividad,actividades,activos,acto,actor,actores,actos,actual,actuales,actualidad,actualización,actualmente,actuar,actúa,acuerdo,acusa,acusaciones,...,únete,única,único,–,‘,’,“,”,•,‼️,♥️♥️♥️😃⭐,⚠️,❤️,➡️,🇪🇸,🇵🇪,🇵🇪httptmemovimientoqanonperu,🇺🇸,🍿,👀,👆,👇,👉,👉httpbitlyjzitic,👉🏻,👍,👍👍👍,👏👏👏,💉,💪,💪💪,🔥,🔴,😁,😂,😂😂,😂😂😂,😃,😉,😊,😔,😡,😱,😳,🙄,🙏,🙏🏻,🚨,🤔,🤣
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71187,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [60]:
from sklearn.cluster import KMeans
number_of_clusters=5
km = KMeans(n_clusters=number_of_clusters)
km.fit(matrix)
km.fit

<bound method KMeans.fit of KMeans(n_clusters=5)>

In [61]:
print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(number_of_clusters):
    top_ten_words = [terms[ind] for ind in order_centroids[i, :15]]
    print("Cluster {}: {}".format(i, ' '.join(top_ten_words)))

Top terms per cluster:
Cluster 0: gracias trump video así the 👇 verdad gente vacuna ahora grupo covid ser 👍 hola
Cluster 1: si alguien verdad así puede ver pues gente trump solo ser vacuna sabe ahora saber
Cluster 2: q anonperú 🇵🇪 grupo anon si mapas trump creo mas x peru perú solo d
Cluster 3: día buen cada hoy gracias dia buenos días q trump si gente patriotas mañana hola
Cluster 4: anon httpstmemovimientoqanonperu 🇵🇪 perú únete q ganado luz peru 🇵🇪httptmemovimientoqanonperu httpstmemovimientoqanonpe vacunas si niños covid


Trying to recognize entities in spanish with Spacy:

In [62]:
!python -m spacy download es_core_news_sm

You should consider upgrading via the '/Users/biancapallaro/.pyenv/versions/3.8.2/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [63]:
#!pip install spacy
import pandas as pd
import spacy
import requests

nlp = spacy.load("es_core_news_sm")

pd.set_option("display.max_rows", 200)

In [64]:
text = '\n'.join(df_no_sm.message.values)

In [ ]:
nlp.max_length = 13578481
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [ ]:
entities = [(ent.text, ent.label_, ent.lemma_) for ent in doc.ents]
df_entity = pd.DataFrame(entities, columns=['text', 'type', 'lemma'])
df_entity.head(25)

In [ ]:
df_entity[df_entity.type == 'PER'].lemma.value_counts().head(30)

In [ ]:
df_entity[df_entity.type == 'ORG'].lemma.value_counts().head(30)

In [ ]:
df_entity[df_entity.type == 'LOC'].lemma.value_counts().head(30)

In [ ]:
df_entity[df_entity.type == 'MISC'].lemma.value_counts().head(30)